In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from random import randint
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers, regularizers

# Load the TensorBoard notebook extension
%load_ext tensorboard
import datetime

In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [4]:
d = mnist
(x_train, y_train), (x_test, y_test) = d.load_data()

x_train = x_train.reshape(60000, 28,28,1)
x_test = x_test.reshape(10000, 28,28,1)

### Create a CNN Model

In [5]:
def my_model():
    inputs = keras.Input(shape=(28,28,1))
    x = layers.Conv2D(32, 5, padding = 'valid', kernel_regularizer = regularizers.l2(0.01),)(inputs)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    ## x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 3,  padding = 'valid', kernel_regularizer = regularizers.l2(0.01),)(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)   
    ## x = layers.MaxPooling2D()(x)                                                                                
    x = layers.Conv2D(128, 3,  padding = 'valid', kernel_regularizer = regularizers.l2(0.01),)(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation = 'relu', kernel_regularizer = regularizers.l2(0.01))(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(32, activation = 'relu', kernel_regularizer = regularizers.l2(0.01))(x)
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(10)(x)
    model = keras.Model(inputs = inputs, outputs = outputs)
    return model

In [6]:
model = my_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 24, 24, 32)        832       
                                                                 
 batch_normalization (BatchN  (None, 24, 24, 32)       128       
 ormalization)                                                   
                                                                 
 tf.nn.relu (TFOpLambda)     (None, 24, 24, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 64)        18496     
                                                                 
 batch_normalization_1 (Batc  (None, 22, 22, 64)       256       
 hNormalization)                                             

In [7]:
model.compile(
              loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer = keras.optimizers.SGD(learning_rate = 0.001),
              metrics = ['accuracy']
              )

In [8]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [8]:
history = model.fit(x_train, y_train, validation_split=0.4, validation_data=(x_test,y_test), batch_size = 64, epochs = 2500, verbose = 2, callbacks=[tensorboard_callback])

Epoch 1/2500
563/563 - 31s - loss: 4.0871 - accuracy: 0.6245 - val_loss: 3.2575 - val_accuracy: 0.9290 - 31s/epoch - 55ms/step
Epoch 2/2500
563/563 - 15s - loss: 3.4633 - accuracy: 0.8285 - val_loss: 3.0717 - val_accuracy: 0.9509 - 15s/epoch - 27ms/step
Epoch 3/2500
563/563 - 15s - loss: 3.2533 - accuracy: 0.8763 - val_loss: 2.9591 - val_accuracy: 0.9614 - 15s/epoch - 27ms/step
Epoch 4/2500
563/563 - 15s - loss: 3.1278 - accuracy: 0.8957 - val_loss: 2.8786 - val_accuracy: 0.9664 - 15s/epoch - 27ms/step
Epoch 5/2500
563/563 - 15s - loss: 3.0070 - accuracy: 0.9159 - val_loss: 2.8046 - val_accuracy: 0.9691 - 15s/epoch - 26ms/step
Epoch 6/2500
563/563 - 15s - loss: 2.9164 - accuracy: 0.9279 - val_loss: 2.7328 - val_accuracy: 0.9732 - 15s/epoch - 27ms/step
Epoch 7/2500
563/563 - 15s - loss: 2.8354 - accuracy: 0.9345 - val_loss: 2.6698 - val_accuracy: 0.9743 - 15s/epoch - 27ms/step
Epoch 8/2500
563/563 - 15s - loss: 2.7612 - accuracy: 0.9388 - val_loss: 2.6078 - val_accuracy: 0.9759 - 15s/ep

In [ ]:
%tensorboard --logdir logs/fit

##### Saving the Entire Model

model.save()

In [9]:
import os.path
if os.path.isfile('ANN/18-04-22 DL Practical High Epoch.h5') is False:
    model.save('ANN/18-04-22 DL Practical High Epoch.h5')

In [10]:
from tensorflow.keras.models import load_model
new_model = load_model('ANN/18-04-22 DL Practical High Epoch.h5')

In [11]:
new_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 24, 24, 32)        832       
                                                                 
 batch_normalization (BatchN  (None, 24, 24, 32)       128       
 ormalization)                                                   
                                                                 
 tf.nn.relu (TFOpLambda)     (None, 24, 24, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 64)        18496     
                                                                 
 batch_normalization_1 (Batc  (None, 22, 22, 64)       256       
 hNormalization)                                             

In [12]:
new_model.get_weights()

[array([[[[-4.81212288e-02, -8.22246075e-05,  9.13801044e-03,
           -4.99367304e-02,  4.14807200e-02,  4.77290004e-02,
           -5.38181253e-02, -1.75691694e-02,  2.31558606e-02,
            7.56615549e-02,  1.38204023e-02,  4.34775352e-02,
            2.64655948e-02, -5.61488904e-02, -2.23349631e-02,
           -8.18885565e-02, -3.85312065e-02,  3.43166888e-02,
           -5.29997870e-02, -3.47024947e-02,  4.64204699e-03,
            3.76420543e-02,  4.73470092e-02, -6.43281713e-02,
            3.98823321e-02, -7.39681497e-02,  2.06949562e-03,
           -4.22198921e-02, -7.07334429e-02, -4.90971431e-02,
           -6.51005358e-02,  6.86842501e-02]],
 
         [[ 1.31396502e-02, -3.44920941e-02,  2.26302072e-02,
           -6.12094849e-02,  2.29806826e-02, -7.03020915e-02,
            2.93050706e-02,  7.74598569e-02,  8.16766024e-02,
           -2.51672417e-03, -5.24085201e-02,  7.11800903e-02,
           -7.29691163e-02, -3.61660607e-02,  4.96810526e-02,
            1.3588167

In [13]:
new_model.optimizer

Saving only the weights

In [14]:
if os.path.isfile('ANN/MNIST weights.h5') is False:
    model.save_weights('ANN/MNIST weights.h5')